In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from config import TRAIN_WAV_DIR,DCASE_CSV_DIR_TRAIN
import pathlib
from tqdm import tqdm
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input as res_in
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import librosa
from PIL import Image
import cv2

Using TensorFlow backend.
d:\projects\dcase\venv\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
d:\projects\dcase\venv\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
base_model = InceptionResNetV2(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.output)

In [3]:
from keras.backend import set_image_data_format
set_image_data_format('channels_last')
directory = TRAIN_WAV_DIR
files = pathlib.Path(directory)
files = list(files.glob('*.wav'))
for num,wav in tqdm(enumerate(files),total=len(files)):
    name=wav.stem.split('_')[0]
    id_wav=int(wav.stem.split('_')[2])
    num_id_wav=int(wav.stem.split('_')[3])
    if name=='normal':
        y=1
    else:
        y=0

    sig, sr =librosa.load(wav, sr=None, mono=True)
    mel_spectrogram = librosa.feature.melspectrogram(y=sig,
                                                     sr=sr,
                                                     n_fft=1024,
                                                     hop_length=512,)
    img = Image.fromarray(mel_spectrogram,mode='RGB')
    imgUMat = np.float32(img)
    x = cv2.resize(imgUMat, dsize=(299, 299), interpolation=cv2.INTER_CUBIC)
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = res_in(x)
    flatten = model.predict(x)
    if num==0:
        feat_name=[str(i) for i in range(flatten.shape[1])]
        col=feat_name+['y']+['ind']+['num']
        data=np.zeros((len(files),len(col)))
        X_b=pd.DataFrame(data)
        X_b.columns=col
    X_b.loc[num,feat_name]=flatten
    X_b.loc[num,'y']=y
    X_b.loc[num,'ind']=id_wav
    X_b.loc[num,'num']=num_id_wav

100%|██████████| 3675/3675 [04:25<00:00, 13.84it/s]


In [4]:
print(DCASE_CSV_DIR_TRAIN)
X_b.to_csv(DCASE_CSV_DIR_TRAIN+'Xtrain.csv')

D:\Projects\dcase\thrash\csv_incepcion_features\train\


In [7]:
from config import TEST_WAV_DIR,DCASE_CSV_DIR_TEST
directory = TEST_WAV_DIR
files = pathlib.Path(directory)
files = list(files.glob('*.wav'))
count_good=0
count_bad=0
for num,wav in tqdm(enumerate(files),total=len(files)):
    name=wav.stem.split('_')[0]
    id_wav=int(wav.stem.split('_')[2])
    num_id_wav=int(wav.stem.split('_')[3])
    if name=='normal':
        y=1
    else:
        y=0
    sig, sr =librosa.load(wav, sr=None, mono=True)
    mel_spectrogram = librosa.feature.melspectrogram(y=sig,
                                                     sr=sr,
                                                     n_fft=1024,
                                                     hop_length=512,)
    img = Image.fromarray(mel_spectrogram,mode='RGB')
    imgUMat = np.float32(img)
    x = cv2.resize(imgUMat, dsize=(299, 299), interpolation=cv2.INTER_CUBIC)
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = res_in(x)
    flatten = model.predict(x)
    if num==0:
        feat_name=[str(i) for i in range(flatten.shape[1])]
        col=feat_name+['y']+['ind']+['num']
        data=np.zeros((len(files),len(col)))
        X_b=pd.DataFrame(data)
        X_b.columns=col
    X_b.loc[num,feat_name]=flatten
    X_b.loc[num,'y']=y
    X_b.loc[num,'ind']=id_wav
    X_b.loc[num,'num']=num_id_wav

100%|██████████| 1875/1875 [01:58<00:00, 15.76it/s]


In [8]:
X_b.to_csv(DCASE_CSV_DIR_TEST+'Xtest.csv')